In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
import keras
import matplotlib.pyplot as plt
import tensorflow as tf 
import cv2
import os


input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

initial_mode2 = keras.initializers.he_normal(seed=None)
initial_mode = keras.initializers.glorot_normal(seed=None)

for i in range(1,2):
    ## Build the Convolution autoencoders model

    x = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer=initial_mode)(input_img)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer=initial_mode)(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer=initial_mode)(x)
    encoded = x

    # at this point the representation is (4, 4, 8) i.e. 128-dimensional

    x = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer=initial_mode)(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer=initial_mode)(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same',kernel_initializer=initial_mode2)(x)



    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

    from keras.datasets import mnist
    import numpy as np

    (x_train, _), (x_test, _) = mnist.load_data()

    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
    x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

    from keras.callbacks import TensorBoard

    
    

    class CustomSaver(tf.keras.callbacks.Callback):
        def on_epoch_begin(self, epoch, logs={}):            
            loop = epoch + 1
            
            if loop%5 == 0:  # or save after some epoch, each k-th epoch etc.
                print('Save the image')
                decoded_imgs = autoencoder.predict(x_test)
                img = decoded_imgs[0]
                img = img.reshape(28,28)
                
                # save the img
                current_dir = os.getcwd()
                plt.imshow(img,'gray')
                plt.savefig(current_dir+'/Decoded_img/Decoded_image_epoch'+str(loop)+'.png')


    saver = CustomSaver()
    current_dir = os.getcwd()

    result = autoencoder.fit(x_train, x_train,
                                        epochs=50,
                                        batch_size=128,
                                        shuffle=True,
                                        validation_data=(x_test, x_test),
                                        callbacks=[saver,TensorBoard(log_dir=current_dir+'/tensorboard_data/New_lecun')])
    
    


#print(autoencoder.summary())




#decoded_images = autoencoder.predict(x_test)

In [ ]:
autoencoder.save('New_CAE_glorot.h5')